In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from mnist import MNIST
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 数据读取
mndata = MNIST('./data')
train_valid_images, train_valid_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()
train_valid_images, train_valid_labels = np.array(train_valid_images), np.array(train_valid_labels)
test_images, test_labels = np.array(test_images), np.array(test_labels)

In [3]:
train_images, valid_images, train_labels, valid_labels = train_test_split(train_valid_images, train_valid_labels, test_size=0.3, random_state=0, stratify=train_valid_labels)

In [4]:
# 模型训练，两层全连接神经网络
# 激活函数
# 反向传播，loss以及梯度的计算
# 学习率下降策略
# L2正则化
# 优化器SGD
# 保存模型

# 模型使用 Linear -> ReLu -> Linear -> CrossEntropy -> SGD

class Linear():
    def __init__(self, input_dim, output_dim, bias=True):
        self._gradient = None
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.bias = bias

        self.w = np.random.uniform(-np.sqrt(1 / self.input_dim), np.sqrt(1 / self.input_dim), [input_dim, output_dim]) # input_dim * output_dim
        self.b = np.random.uniform(-np.sqrt(1 / self.input_dim), np.sqrt(1 / self.input_dim), [1, output_dim])
        if bias:
            self.w = np.vstack((self.w, self.b.reshape(1, -1)))
        self._parameters_dict = None

    def __call__(self, X, update_grad=False):
        num_sample, num_feature = X.shape
        if self.bias:
            num_feature += 1
            X = np.hstack((X, np.ones(num_sample).reshape(-1, 1))) # num_sample * input_dim
        
        if update_grad:
            self._update_gradient(grad_val=X.T)
        return np.dot(X, self.w) # num_sample * output_dim
    
    def _update_gradient(self, grad_val):
        self._gradient = grad_val

    @property
    def gradient(self):
        return self._gradient # input_dim * num_samples

    def _load_params(self, params):
        self.w = params['w'].data.numpy().T
        self.b = params['b'].data.numpy()
        if self.bias:
            self.w = np.vstack((self.w, self.b.reshape(1, -1)))

    @property
    def parameters(self):
        if self.bias:
            self._parameters_dict = {'w': self.w[:-1, :], 'b': self.w[-1, :]}
        else:
            self._parameters_dict = {'w': self.w}
        return self._parameters_dict
    
    def _update_params(self, update_value):
        self.w -= update_value

    def load(self, params_dict):
        self.w = params_dict['w']
        self.b = params_dict['b']
        if self.bias:
            self.w = np.vstack((self.w, self.b.reshape(1, -1)))
            
class ReLu():
    def __init__(self, inplace=False):
        self.inplace = inplace
        self._gradient = None
    
    def __call__(self, X, update_grad=False):
        if update_grad:
            self._update_gradient(grad_val=(X > 0))
        if self.inplace:
            np.maximum(X, 0, X)
        else:
            X = X.copy()
            return np.maximum(X, 0, X)
    
    def _update_gradient(self, grad_val):
        self._gradient = grad_val

    @property
    def gradient(self):
        return self._gradient

class LeakyReLu():
    def __init__(self, negative_slpoe=0.1, inplace=False):
        self.negative_slope = negative_slpoe
        self.inplace = inplace
        self._gradient = None
    
    def __call__(self, X, update_grad=False):
        if not self.inplace:
            X = X.copy()
        X[X > 0] = 1
        X[X < 0] = -self.negative_slope
        if update_grad:
            self._update_gradient(grad_val=X)
        if not self.inplace:
            return X

    def _update_gradient(self, grad_val):
        self._gradient = grad_val
    
    @property
    def gradient(self):
        return self._gradient
    
    def _train(self):
        self._train_mod = True
    
    def _eval(self):
        self._train_mod = False
        
class CrossEntropLoss():
    def __init__(self):
       self._gradient = None
       self._train_mod = False

    def __call__(self, predicts, labels):
        num_classes = predicts.shape[1]

        transform_labels = np.eye(num_classes)[labels]
        if num_classes > 2:
            transform_predicts = np.exp(predicts) / np.exp(predicts).sum(axis=1, keepdims=1)
        else:
            raise NotImplementedError()
        
        res = -transform_labels * np.log(transform_predicts)
        if self._train_mod:
            self._update_gradient(grad_val=transform_predicts - transform_labels)
        return np.sum(res) / res.shape[0]

    def _update_gradient(self, grad_val):
        self._gradient = grad_val

    @property
    def gradient(self):
        return self._gradient # num_samples * num_class

    @property
    def train_mod(self):
        return self._train_mod
    
    @train_mod.setter
    def train_mod(self, flag):
        self._train_mod = flag

class Model():
    def __init__(self, input_dim, hidden_dim, output_dim, loss_func):
        self.layer1 = Linear(input_dim, hidden_dim)
        self.relu = LeakyReLu(negative_slpoe=0.1)
        self.layer2 = Linear(hidden_dim, output_dim)
        self.loss_func = loss_func
        self._parameters_dict = self.parameters
        self._train_mod = False
    
    def __call__(self, X):
        X = self.layer1(X, update_grad=self._train_mod)
        X = self.relu(X, update_grad=self._train_mod)
        X = self.layer2(X, update_grad=self._train_mod)
        return X

    @property
    def parameters(self):
        self._parameters_dict = dict((k, v.parameters) for (k, v) in self.__dict__.items() if hasattr(v, 'parameters'))
        return self._parameters_dict
    
    # 当前版本未能实现计算图，所以需要手动编辑梯度
    def backward(self, l2_weight=0):
        self.gradient_dict = dict()
        if self.layer2.bias:
            self.gradient_dict['layer1'] = np.dot(self.layer1.gradient, self.relu.gradient * np.dot(self.loss_func.gradient, self.layer2.w[:-1, :].T))
        else:
            self.gradient_dict['layer1'] = np.dot(self.layer1.gradient, self.relu.gradient * np.dot(self.loss_func.gradient, self.layer2.w.T))
        self.gradient_dict['layer2'] = np.dot(self.layer2.gradient, self.loss_func.gradient)

        if self.layer1.bias:
            self.gradient_dict['layer1'][:-1, :] += l2_weight * self.layer1.w[:-1, :]
        else:
            self.gradient_dict['layer1'] += l2_weight * self.layer1.w
        
        if self.layer2.bias:
            self.gradient_dict['layer2'][:-1, :] += l2_weight * self.layer2.w[:-1, :]
        else:
            self.gradient_dict['layer2'] += l2_weight * self.layer2.w
    
    def sgd_step(self, lr=0.01, decay_rate=0.98, decay_step=1000, cur_step=0):
        # 使用指数化的步长缩减策略
        cur_lr = lr * decay_rate ** (cur_step / decay_step)
        for layer, params in self.gradient_dict.items():
            self.__dict__[layer]._update_params(cur_lr * params)
    
    def dump(self, path=None):
        if path is None:
            cur_path = os.getcwd()
            file_path = os.path.join(cur_path, 'model')
            os.makedirs(file_path, exist_ok=True)
        else:
            file_path = path

        for cur_layer in self.parameters:
            cur_layer_path = os.path.join(file_path, cur_layer)
            os.makedirs(cur_layer_path, exist_ok=True)
            for content in self.parameters[cur_layer]:
                self.parameters[cur_layer][content].dump(os.path.join(cur_layer_path, f'{content}.npy'))
            
    def load(self, path):
        param_dict = {}
        for layer in os.listdir(path):
            param_dict[layer] = dict()
            layer_path = os.path.join(path, layer)
            for content in os.listdir(layer_path):
                content_name = content.split('.')[0]
                layer_content = np.load(os.path.join(layer_path, content), allow_pickle=True)
                param_dict[layer][content_name] = layer_content
        for layer in param_dict:
            self.__dict__[layer].load(param_dict[layer])
    
    def train(self):
        self._train_mod = True
        self.loss_func.train_mod = True
    
    def eval(self):
        self._train_mod = False
        self.loss_func.train_mod = False

In [5]:
loss_func = CrossEntropLoss()

m = Model(input_dim=784, hidden_dim=64, output_dim=10, loss_func=loss_func)
train_index = [_ for _ in range(len(train_images))]
train_acc_list = []
valid_acc_list = []
test_acc_list = []

train_loss_list = []
valid_loss_list = []
test_loss_list = []
for epoch in tqdm(range(20000)):
    m.train()
    idx = np.random.choice(train_index)
    input_x = train_images[idx].reshape(1, -1)
    labels = train_labels[idx]
    predicts = m(input_x)
    
    train_loss = loss_func(predicts, labels)
    train_loss_list.append(train_loss)

    m.eval()
    valid_predict = m(valid_images)
    test_predict = m(test_images)

    valid_loss = loss_func(valid_predict, valid_labels)
    test_loss = loss_func(test_predict, test_labels)
    valid_loss_list.append(valid_loss)
    test_loss_list.append(test_loss)

    valid_acc = accuracy_score(valid_predict.argmax(axis=1), valid_labels)
    test_acc = accuracy_score(test_predict.argmax(axis=1), test_labels)
    valid_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)
    
    m.backward(l2_weight=0.0001)
    m.sgd_step(lr=0.05, cur_step=epoch, decay_step=500)

100%|██████████| 20000/20000 [28:52<00:00, 11.54it/s]


In [19]:
def calc_metrics(model, dataset, label):
    predicts = model(dataset).argmax(axis=1)
    acc = accuracy_score(predicts, label)
    prec = precision_score(predicts, label, average='macro')
    rec = recall_score(predicts, label, average='macro')
    f1 = f1_score(predicts, label, average='macro')
    print(f"Accuracy: {acc:.4f} Precision: {prec:.4f} Recall: {rec:.4f} F1-Score: {f1:.4f}")

In [20]:
calc_metrics(m, train_images, train_labels)
calc_metrics(m, valid_images, valid_labels)
calc_metrics(m, test_images, test_labels)

Accuracy: 0.8732 Precision: 0.8720 Recall: 0.8733 F1-Score: 0.8721
Accuracy: 0.8732 Precision: 0.8718 Recall: 0.8731 F1-Score: 0.8719
Accuracy: 0.8808 Precision: 0.8797 Recall: 0.8808 F1-Score: 0.8796


In [21]:
m.dump()

In [ ]:
# 参数搜索
# 首先对模型的训练过程进行封装
def train_one_model()

In [22]:
copy_m = Model(input_dim=784, hidden_dim=64, output_dim=10, loss_func=loss_func)

In [23]:
copy_m.load('/Users/tsuilai/Desktop/课程代码/神经网络与深度学习/Assign1/model')

In [24]:
copy_test_predicts = copy_m(test_images).argmax(axis=1)
np.sum(copy_test_predicts == test_labels) / len(test_labels)

0.8808